# Comprobaciones de versiones instaladas

In [1]:
!python --version

Python 3.8.13


In [2]:
!java -version

/usr/bin/sh: 1: java: not found


In [3]:
!find / -iname spark*.jar 2>/dev/null|grep sql

/home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/pyspark/jars/spark-sql_2.12-3.3.1.jar


In [4]:
import os

# print(os.environ["AWS_REGION"])
# os.environ["AWS_SECRET_ACCESS_KEY"]
# os.environ["AWS_ACCESS_KEY_ID"]

# Obtener todas las variables de entorno
env_variables = os.environ

# Imprimir todas las variables de entorno de forma legible
for key, value in env_variables.items():
    if "AWS" in key:
        print(f"{key}: {value}")


AWS_REGION: us-east-1
AWS_SECRET_ACCESS_KEY: t4bl4red0nd4
AWS_ACCESS_KEY_ID: admin


In [5]:
!pip install pyspark==3.3.1

# Iniciar conexión spark

In [6]:
import os
import pandas as pd
import numpy as np
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, FloatType, BooleanType
from pyspark.sql.types import DoubleType, IntegerType, StringType
from pyspark import SQLContext

#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.iceberg:iceberg-spark-runtime-3.2_2.12:1.1.0,org.mariadb.jdbc:mariadb-java-client:2.3.0,org.postgresql:postgresql:42.5.4 pyspark-shell'
dst_lh_url  = os.environ.get('DST_LH_URL', 'spark://spark-master-0.spark-headless.demml.svc.cluster.local:7077')
dst_lh_appn = os.environ.get('DST_LH_APPN', 'conexion_pyspark_iceberg_edu')
import subprocess
my_pod_ip = subprocess.run(['hostname', '-I'], stdout=subprocess.PIPE).stdout.decode('utf-8').strip(' \n\t')
my_pod_ip


'172.21.23.248'

In [8]:
spark = SparkSession.builder \
    .master(dst_lh_url) \
    .appName(dst_lh_appn) \
    .config('spark.jars.packages','org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.1.0,org.mariadb.jdbc:mariadb-java-client:2.3.0,org.postgresql:postgresql:42.5.4,software.amazon.awssdk:bundle:2.17.257,software.amazon.awssdk:url-connection-client:2.17.257,software.amazon.awssdk:s3:2.17.257,software.amazon.awssdk:iam:2.17.257,org.apache.hadoop:hadoop-aws:3.2.3') \
    .config('spark.sql.extensions','org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions') \
    .config('spark.sql.catalog.spark_catalog','org.apache.iceberg.spark.SparkSessionCatalog') \
    .config('spark.sql.catalog.spark_catalog.catalog-impl','org.apache.iceberg.jdbc.JdbcCatalog') \
    .config('spark.sql.catalog.spark_catalog.uri','jdbc:postgresql://postgresql:5432/iceberg') \
    .config('spark.driver.host', my_pod_ip) \
    .config('spark.sql.catalog.spark_catalog.jdbc.useSSL','false') \
    .config('spark.sql.catalog.spark_catalog.jdbc.user','iceberg') \
    .config('spark.sql.catalog.spark_catalog.jdbc.password','iceberg') \
    .config('spark.sql.catalog.spark_catalog.warehouse','s3a://warehouse/') \
    .config('spark.sql.catalog.spark_catalog.s3.endpoint','http://minio:9000') \
    .config('spark.sql.catalog.spark_catalog.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO') \
    .config('spark.sql.defaultCatalog','spark_catalog') \
    .config('spark.hadoop.fs.s3a.endpoint','http://minio:9000') \
    .config('spark.hadoop.fs.s3a.access.key','admin') \
    .config('spark.hadoop.fs.s3a.secret.key','t4bl4red0nd4') \
    .config('spark.hadoop.fs.s3a.path.style.access','true') \
    .config('spark.hadoop.fs.s3a.impl','org.apache.hadoop.fs.s3a.S3AFileSystem') \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .getOrCreate()
spark

:: loading settings :: url = jar:file:/home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/coder/.ivy2/cache
The jars for the packages stored in: /home/coder/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.3_2.12 added as a dependency
org.mariadb.jdbc#mariadb-java-client added as a dependency
org.postgresql#postgresql added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
software.amazon.awssdk#s3 added as a dependency
software.amazon.awssdk#iam added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c28f4f0a-01a7-4cbc-9006-57c027f6e75c;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.3_2.12;1.1.0 in central
	found org.mariadb.jdbc#mariadb-java-client;2.3.0 in central
	found org.postgresql#postgresql;42.5.4 in central
	found org.checkerframework#checker-qual;3.5.0 in central
	found software.amazon.awssdk#bundle;2.17.257 in central
	found software.am

23/09/19 06:14:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [9]:
spark.sql("show current namespace").show() 

+-------------+---------+
|      catalog|namespace|
+-------------+---------+
|spark_catalog|  default|
+-------------+---------+



In [10]:
spark.sql("show databases").show() 

+---------+
|namespace|
+---------+
|  default|
+---------+



In [11]:
spark.sql("show tables").show() 

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



# Creación de tabla desde spark en minio (Iceberg)

!!! NO borrar siguiente celda comentada ¡¡¡

In [16]:
# spark.sql("CREATE TABLE default.IRIS_EDU_V5 (sepal_length double, sepal_width double, petal_length double, petal_width double, variety string, target int) USING iceberg OPTIONS ('write.object-storage.enabled'='true', 'write.data-path'='s3://warehouse/', 'write.format'='parquet');")

In [12]:
spark.table("spark_catalog.default.IRIS_EDU_V5").show(5)
# spark.sql("SELECT * FROM spark_catalog.default.IRIS_EDU_V5").show()

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


+------------+-----------+------------+-----------+-------+------+
|sepal_length|sepal_width|petal_length|petal_width|variety|target|
+------------+-----------+------------+-----------+-------+------+
|         5.1|        3.5|         1.4|        0.2| setosa|     0|
|         4.9|        3.0|         1.4|        0.2| setosa|     0|
|         4.7|        3.2|         1.3|        0.2| setosa|     0|
|         4.6|        3.1|         1.5|        0.2| setosa|     0|
|         5.0|        3.6|         1.4|        0.2| setosa|     0|
+------------+-----------+------------+-----------+-------+------+
only showing top 5 rows



# Creo un dataframe de pyspark a partir de un dataframe de pandas y lo escribo en iceberg (se puede ver en minio) 

In [ ]:
import pandas as pd
from sklearn.datasets import load_iris

iris = load_iris()

df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['variety'] = iris.target_names[iris.target]
df['target'] = iris.target

df.columns = ['sepal_length','sepal_width','petal_length','petal_width','variety','target']

dfs = spark.createDataFrame(df)
dfs.show(5)

# Creo datos sinteéticos en base a la carga inicial de iris

In [33]:
for i in range(1000):
    pivot = df.sample(10000,replace=True)
    pivot_s = spark.createDataFrame(pivot)
    pivot_s.write.format("iceberg").mode("append").save("default.IRIS_EDU_V5")
    # pivot_s.show(5)

In [34]:
df_size = spark.table("spark_catalog.default.IRIS_EDU_V5")

df_size.count()


15520150

In [16]:
# dfs.write.format("iceberg").mode("append").save("default.IRIS_EDU_V5")

In [35]:
%load_ext kedro.extras.extensions.ipython
%reload_kedro

catalog.list()
# catalog.load('data_raw_minio')
# catalog.load('my_iceberg_table').show()

[09/19/23 07:06:24] INFO     Resolved project path as: /home/coder/eromero/iris-pyspark.            ]8;id=221258;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=535842;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py#139\139]8;;\
                             To set a different path, run '%reload_kedro <project_root>'                           

[09/19/23 07:06:24] WARNING  /home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/io ]8;id=803600;file:///home/coder/miniconda3/envs/demml/lib/python3.8/warnings.py\warnings.py]8;;\:]8;id=350424;file:///home/coder/miniconda3/envs/demml/lib/python3.8/warnings.py#109\109]8;;\
                             /data_catalog.py:300: FutureWarning: Defining the 'layer' attribute at                
                             the top level is deprecated and will be removed in Kedro 0.19.0.                      
                             Please move 'layer' inside the 'metadata' -> 'kedro-viz' attributes.                  
                             See                                                                                   
                             https://docs.kedro.org/en/latest/visualisation/kedro-viz_visualisation                
                             .html#visualise-layers for more information.                                          
                               warnings.warn(                                                                      
                                                                                                                   

[09/19/23 07:06:25] INFO     Kedro project iris-pyspark                                             ]8;id=357665;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=802515;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py#108\108]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=359453;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=26348;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py#109\109]8;;\
                             'pipelines'                                                                           

[09/19/23 07:06:26] INFO     Registered line magic 'run_viz'                                        ]8;id=226216;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=546536;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py#115\115]8;;\

...

[09/19/23 07:06:27] WARNING  /home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/io ]8;id=872373;file:///home/coder/miniconda3/envs/demml/lib/python3.8/warnings.py\warnings.py]8;;\:]8;id=996436;file:///home/coder/miniconda3/envs/demml/lib/python3.8/warnings.py#109\109]8;;\
                             /data_catalog.py:300: FutureWarning: Defining the 'layer' attribute at                
                             the top level is deprecated and will be removed in Kedro 0.19.0.                      
                             Please move 'layer' inside the 'metadata' -> 'kedro-viz' attributes.                  
                             See                                                                                   
                             https://docs.kedro.org/en/latest/visualisation/kedro-viz_visualisation                
                             .html#visualise-layers for more information.                                          
                               warnings.warn(                                                                      
                                                                                                                   

                    INFO     Kedro project iris-pyspark                                             ]8;id=144042;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=535993;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py#108\108]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=271734;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=802959;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py#109\109]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=325438;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=29001;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/ipython/__init__.py#115\115]8;;\


[
    'data_raw_minio',
    'my_iceberg_table',
    'parameters',
    'params:example_test_data_ratio',
    'params:example_num_trees'
]

In [36]:
catalog.load('my_iceberg_table').show()

[09/19/23 07:06:31] INFO     Loading data from 'my_iceberg_table' (SparkDataSet)...             ]8;id=632888;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=109776;file:///home/coder/miniconda3/envs/demml/lib/python3.8/site-packages/kedro/io/data_catalog.py#492\492]8;;\

23/09/19 07:06:31 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+------------+-----------+------------+-----------+----------+------+
|sepal_length|sepal_width|petal_length|petal_width|   variety|target|
+------------+-----------+------------+-----------+----------+------+
|         5.6|        2.8|         4.9|        2.0| virginica|     2|
|         6.5|        3.0|         5.2|        2.0| virginica|     2|
|         6.2|        2.2|         4.5|        1.5|versicolor|     1|
|         7.2|        3.0|         5.8|        1.6| virginica|     2|
|         5.9|        3.2|         4.8|        1.8|versicolor|     1|
|         5.8|        2.7|         3.9|        1.2|versicolor|     1|
|         5.5|        4.2|         1.4|        0.2|    setosa|     0|
|         6.3|        2.7|         4.9|        1.8| virginica|     2|
|         6.6|        2.9|         4.6|        1.3|versicolor|     1|
|         5.6|        2.7|         4.2|        1.3|versicolor|     1|
|         5.5|        2.4|         3.7|        1.0|versicolor|     1|
|         5.4|      

In [37]:
spark.stop()